<a href="https://colab.research.google.com/github/gorilux/google-colab/blob/main/EuromillionsAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam,Nadam, SGD

2024-11-15 20:35:49.907798: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


2023-09-05 20:35:15.085329: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-05 20:35:15.091118: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-05 20:35:15.091297: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [ ]:
!curl -s 'https://lottery.merseyworld.com/cgi-bin/lottery?days=20&Machine=Z&Ballset=0&order=0&show=1&year=0&display=CSV' 2>&1 | grep -e "^[ 0-9]*, "  -e "^No." > euromillions_results.csv

In [ ]:
df = pd.read_csv('euromillions_results.csv', header = 0, index_col = 0, names = ['No.','Day','DD','MMM', 'YYYY', 'N1', 'N2', 'N3', 'N4', 'N5','L1', 'L2', 'Jackpot', 'Wins'])
#df = pd.read_csv('euromillions_results.csv', header = 0)
#df = pd.read_csv('csv/megamillions.csv', names=['name','month','day','year','s1','s2','s3','s4','s5','s6'], index_col=False, dtype={'day': 'str', 'month': 'str', 'year': 'str'})
#df = df.reindex(index=df.index[::-1])

df.head()
# df.ix[(df['No.'] < 941)]
## Number of starts increased in draw Number  => 941
## if (row[2] == "27" && row[3] == "Sep" && row[4] == "2016")
# X = data[:,0].reshape(-1,1) # make it a 2-D array of size N x D where D = 1
# Y = data[:,1].reshape(-1,1)

,Day,DD,MMM,YYYY,N1,N2,N3,N4,N5,L1,L2,Jackpot,Wins
No.,,,,,,,,,,,,,
1789,Tue,12,Nov,2024,11,12,50,10,22,10,1,114433995,0
1788,Fri,8,Nov,2024,48,2,35,33,42,1,3,103665143,0
1787,Tue,5,Nov,2024,50,8,37,1,19,9,6,91973310,0
1786,Fri,1,Nov,2024,44,7,50,33,38,2,7,82255604,0
1785,Tue,29,Oct,2024,45,30,19,1,2,12,10,71368025,0


In [ ]:
values = df[['N1','N2','N3','N4','N5']].values
values[0]

array([16, 29, 32, 36, 41])

In [ ]:
scaler = StandardScaler().fit(values)
transformed_dataset = scaler.transform(values)
transformed_df = pd.DataFrame(data=transformed_dataset, index=df.index)
transformed_df.head()

,0,1,2,3,4
No.,,,,,
1,1.140159,1.486734,0.773830,0.235519,-0.238656
2,-0.219317,-0.468069,1.580269,1.547430,1.113125
3,0.838053,0.142807,-0.723843,-0.360804,-0.839447
4,-0.672476,-1.201120,0.889035,0.354784,-0.539051
5,0.989106,0.875858,0.313007,1.189636,0.662531


In [ ]:
number_of_rows= values.shape[0]-1 #all rows
window_length = 5  #sliding window
number_of_predictions = 5
number_of_columns = 5
number_of_features =  number_of_columns * number_of_predictions#columns count
print(number_of_rows)
print(number_of_rows - window_length)
print(number_of_rows - window_length - number_of_predictions)


1663
1658
1653


In [ ]:
#train = np.empty([number_of_rows-window_length, window_length, number_of_features], dtype=float)
train = np.empty([number_of_rows-window_length-number_of_predictions, window_length, number_of_columns], dtype=float)
label = np.empty([number_of_rows-window_length-number_of_predictions, number_of_features], dtype=float)
# create minibatches of window_length rows as element in train array
print(train.shape)
print(label.shape)

# print(row_count)
#for i in range(0, number_of_rows-window_length):
for i in range(0, number_of_rows-window_length-number_of_predictions):
    #print(f"train[{i}] = .iloc[{i}:{i+window_length}]")
    #print(f"label[{i}] = .iloc[{i+window_length}:{i+window_length+1}]")
    train[i]=transformed_df.iloc[i:i+window_length, 0: number_of_features]
    #print(transformed_df.iloc[i:i+window_length, 0: number_of_features])

    for j in range(0, number_of_predictions):
      offset = j * number_of_columns
      # print(f"label[{i},{offset}:{offset + number_of_columns}] = {i+window_length+j}:{i+window_length+1+j}, 0:{number_of_columns}")
      label[i,offset:offset + number_of_columns] = transformed_df.iloc[i+window_length+j: i+window_length+1+j, 0:number_of_columns]

    #print(label[i])


(1653, 5, 5)
(1653, 25)


In [ ]:
train.shape

(1653, 5, 5)

In [ ]:
label.shape

(1653, 25)

In [ ]:
# batch_size = 25
# model = Sequential()
# # add 1st layer with 50 nodes/perceptons, each will process (2,6) matrix block
# model.add(LSTM(50,
#            input_shape=(window_length, number_of_features),
#            return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(50,
#            return_sequences=False))
# model.add(Dropout(0.2))
# model.add(Dense(number_of_features))
# model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

model = Sequential()
# add 1st layer with 50 nodes/perceptons, each will process (2,6) matrix block
model.add(Bidirectional(LSTM(500,
           input_shape=(window_length, number_of_columns),
           return_sequences=True)))
model.add(Dropout(0.2))

model.add(Bidirectional(LSTM(500,
           return_sequences=True)))

model.add(Dropout(0.2))

model.add(Bidirectional(LSTM(500,
           return_sequences=True)))

model.add(Bidirectional(LSTM(500,
           return_sequences=False)))

model.add(Dense(90))

model.add(Dense(number_of_features))

model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])
#model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])


2023-09-05 20:39:23.764541: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-05 20:39:23.764751: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-05 20:39:23.764889: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [ ]:
model.fit(train, label, batch_size=32, epochs=300, verbose=1)

Epoch 1/300


AttributeError: ignored

In [ ]:
predict = np.empty([1, window_length, number_of_columns], dtype=int)
reverse_df = df.reindex(index=df.index[::-1])[['N1','N2','N3','N4','N5']]
predict[0] = reverse_df.iloc[1:1+window_length, 0: number_of_columns]
print(predict[0])
scaled_to_predict = scaler.transform(predict[0])
scaled_predicted_output_1 = model.predict(np.array([scaled_to_predict]))
print(scaled_predicted_output_1.reshape(number_of_predictions,number_of_columns))
print(scaler.inverse_transform(scaled_predicted_output_1.reshape(number_of_predictions,number_of_columns)).astype(int))
#for j in range(0, number_of_predictions):
#print(scaler.inverse_transform(scaled_predicted_output_1).astype(int)[0])
#print(scaler.inverse_transform(predict).astype(int))
#print(label[0].reshape(-1,1))
#print(scaler.inverse_transform(label[0].reshape(-1,1)).astype(int))

[[ 2 15 19 35 44]
 [ 3  9 12 26 30]
 [ 8 27 31 46 50]
 [12 20 25 26 27]
 [12 21 35 39 45]]
1/1 [==============================] - 2s 2s/step
[[ 0.51482296  0.6875239   0.9540354  -0.49106273  0.06042064]
 [ 0.12001567 -0.6702933  -0.6013052  -0.5338145  -1.1183908 ]
 [ 0.43127418  0.24116124 -0.17518681 -0.2047162  -0.25702837]
 [-1.1639048  -1.1024064  -1.2581666  -1.3907565   0.3704145 ]
 [-0.6466008   1.1856693   0.5204103   0.38228914  0.6006668 ]]
[[11 22 33 29 42]
 [ 9 11 20 29 35]
 [11 18 23 32 40]
 [ 0  7 14 22 45]
 [ 4 26 29 37 46]]


In [ ]:
model.save('euromillions_model_window_6')

INFO:tensorflow:Assets written to: euromillions_model_window_6/assets


INFO:tensorflow:Assets written to: euromillions_model_window_6/assets


In [ ]:
stars = df[['L1','L2']].values
stars_scaler = StandardScaler().fit(stars)
stars_transformed_dataset = stars_scaler.transform(stars)
stars_transformed_df = pd.DataFrame(data=stars_transformed_dataset, index=df.index)

In [ ]:
number_of_stars = 2
stars_train = np.empty([number_of_rows-window_length, window_length, number_of_stars], dtype=float)

stars_label = np.empty([number_of_rows-window_length, number_of_stars], dtype=float)
# create minibatches of window_length rows as element in train array
#for i in range(0, number_of_rows-window_length):
for i in range(0, number_of_rows-window_length):
    #print(f"train[{i}] = .iloc[{i}:{i+window_length}]")
    #print(f"label[{i}] = .iloc[{i+window_length}:{i+window_length+1}]")
    stars_train[i]=stars_transformed_df.iloc[i:i+window_length, 0: number_of_stars]
    stars_label[i]=stars_transformed_df.iloc[i+window_length: i+window_length+1, 0: number_of_stars]

In [ ]:
# stars_model = Sequential()
# # add 1st layer with 50 nodes/perceptons, each will process (2,6) matrix block
# stars_model.add(LSTM(50,
#            input_shape=(window_length, number_of_stars),
#            return_sequences=True))
# stars_model.add(Dropout(0.2))
# stars_model.add(LSTM(50,
#            return_sequences=False))
# stars_model.add(Dropout(0.2))
# stars_model.add(Dense(number_of_stars))
# stars_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
stars_model = Sequential()
# add 1st layer with 50 nodes/perceptons, each will process (2,6) matrix block
stars_model.add(Bidirectional(LSTM(240,
           input_shape=(window_length, number_of_stars),
           return_sequences=True)))
stars_model.add(Dropout(0.2))

stars_model.add(Bidirectional(LSTM(240,
           return_sequences=True)))
stars_model.add(Dropout(0.2))

stars_model.add(Bidirectional(LSTM(240,
           return_sequences=True)))
stars_model.add(Bidirectional(LSTM(240,
           return_sequences=False)))

stars_model.add(Dense(120))

stars_model.add(Dense(number_of_stars))
stars_model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

In [ ]:
stars_model.fit(stars_train, stars_label, batch_size=100, epochs=1000, verbose=1)

Epoch 1/1000
16/16 [==============================] - 6s 6ms/step - loss: 0.9974 - accuracy: 0.4186
Epoch 2/1000
16/16 [==============================] - 0s 5ms/step - loss: 0.9956 - accuracy: 0.4559
Epoch 3/1000
16/16 [==============================] - 0s 5ms/step - loss: 0.9942 - accuracy: 0.4958
Epoch 4/1000
16/16 [==============================] - 0s 5ms/step - loss: 0.9926 - accuracy: 0.4984
Epoch 5/1000
16/16 [==============================] - 0s 6ms/step - loss: 0.9921 - accuracy: 0.4984
Epoch 6/1000
16/16 [==============================] - 0s 5ms/step - loss: 0.9921 - accuracy: 0.4997
Epoch 7/1000
16/16 [==============================] - 0s 5ms/step - loss: 0.9917 - accuracy: 0.4971
Epoch 8/1000
16/16 [==============================] - 0s 5ms/step - loss: 0.9916 - accuracy: 0.5074
Epoch 9/1000
16/16 [==============================] - 0s 5ms/step - loss: 0.9920 - accuracy: 0.5055
Epoch 10/1000
16/16 [==============================] - 0s 5ms/step - loss: 0.9915 - accuracy: 0.5106

KeyboardInterrupt: ignored

In [ ]:
stars_predict = np.empty([1, window_length, number_of_stars], dtype=int)
stars_reverse_df = df.reindex(index=df.index[::-1])[['L1','L2']]
stars_predict[0] = stars_reverse_df.iloc[0:0+window_length, 0: number_of_stars]
print(stars_predict[0])
stars_scaled_to_predict = stars_scaler.transform(stars_predict[0])
stars_scaled_predicted_output_1 = stars_model.predict(np.array([stars_scaled_to_predict]))
print(stars_scaler.inverse_transform(stars_scaled_predicted_output_1).astype(int)[0])

[[11 12]
 [ 2  3]]
1/1 [==============================] - 1s 1s/step
[4 7]
